### Import Necessary Libraries

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
import zipfile as zf
import os
import random
import cv2
import imutils



from skimage import feature

from tensorflow.keras.utils import to_categorical
from imutils import build_montages,paths


sns.set()

In [2]:
os.getcwd()

'C:\\Users\\91755'

### Load Train and Test data

In [3]:
wave_train_healthy = os.listdir('C:\\IBM PROJECT\\Dataset\\wave\\training\\healthy\\')
wave_train_park = os.listdir('C:\\IBM PROJECT\\Dataset\\wave\\training\\parkinson\\')

fp_wave_train_healthy = 'C:\\IBM PROJECT\\Dataset\\wave\\training\\healthy\\'
fp_wave_train_park = 'C:\\IBM PROJECT\\Dataset\\wave\\training\\parkinson\\'

wave_test_healthy = os.listdir('C:\\IBM PROJECT\\Dataset\\wave\\testing\\healthy\\')
wave_test_park = os.listdir('C:\\IBM PROJECT\Dataset\\wave\\testing\\parkinson\\')

fp_wave_test_healthy = 'C:\\IBM PROJECT\\Dataset\\wave\\testing\\healthy\\'
fp_wave_test_park = 'C:\\IBM PROJECT\\Dataset\\wave\\testing\\parkinson\\'

#### Function to quantify image

In [4]:
def quantify_image(image):
  features = feature.hog(image,orientations=9,
                pixels_per_cell=(10,10),cells_per_block=(2,2),transform_sqrt=True,block_norm="L1")

  return features

In [5]:
trainX = []
testX = []
outputs = []
trainY = []
testY = []

### Preprocessing

In [6]:
for i in wave_train_healthy:
  image = cv2.imread(fp_wave_train_healthy+i)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image =cv2.threshold(image, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  trainX.append(features)
  trainY.append('healthy')

for i in wave_test_healthy:
  image = cv2.imread(fp_wave_test_healthy+i)
  outputs.append(image)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image = cv2.threshold(image ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  testX.append(features)
  testY.append('healthy')

In [7]:
for i in wave_train_park:
  image = cv2.imread(fp_wave_train_park+i)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image = cv2.threshold(image ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  trainX.append(features)
  trainY.append('parkinson')

for i in wave_test_park:
  image = cv2.imread(fp_wave_test_park+i)
  outputs.append(image)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image = cv2.threshold(image ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  testX.append(features)
  testY.append('parkinson')

In [8]:
trainX = np.array(trainX)
testX = np.array(testX)
trainY = np.array(trainY)
testY = np.array(testY)

In [9]:
print(trainX)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.24999804 0.         0.         ... 0.         0.         0.        ]
 [0.12675901 0.         0.07381472 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [10]:
print(testX)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
print(trainY)

['healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson']


In [12]:
print(testY)

['healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson' 'parkinson'
 'parkinson' 'parkinson' 'parkinson' 'parkinson']


In [13]:
le = LabelEncoder()

In [14]:
trainY = le.fit_transform(trainY)
testY = le.transform(testY)


In [15]:
print(trainY)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [16]:
print(testY)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
